In [1]:
import pandas as pd
import os
import re

def read_and_split_by_device_and_location(folder_path):
    """
    Reads all .xlsx files from a folder, extracts device and location info from filenames,
    adds 'source_file', 'device', and 'location' columns,
    and returns a dictionary mapping device names to their corresponding DataFrames.

    Parameters:
    - folder_path (str): Path to folder containing .xlsx files.

    Returns:
    - dict: Dictionary where keys are device names and values are DataFrames.
    """
    files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]
    df_list = []
    
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        
        # Extract device name (inside parentheses)
        device_match = re.search(r'\((.*?)\)', file)
        device = device_match.group(1) if device_match else "Unknown_Device"
        
        # Extract location (after parentheses and space)
        location_match = re.search(r'\)\s*(.*)\.xlsx$', file)
        location = location_match.group(1) if location_match else "Unknown_Location"
        
        df['device'] = device
        df['location'] = location
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Split the big DataFrame into one per device
    device_dfs = {}
    for device_name, group_df in combined_df.groupby('device'):
        device_dfs[device_name] = group_df.reset_index(drop=True)
    
    return device_dfs

# Example usage
folder_path = "../Data/raw"
device_dataframes = read_and_split_by_device_and_location(folder_path)

# Save each device DataFrame to a separate CSV
output_folder = "../Data/process/"
os.makedirs(output_folder, exist_ok=True)

for device, df in device_dataframes.items():
    safe_device_name = device.replace("/", "_").replace("\\", "_")
    output_csv = os.path.join(output_folder, f"{safe_device_name}.csv")
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"Saved {output_csv}")


Saved ../Data/process/1#High-Temp Fan.csv
Saved ../Data/process/8#Belt Conveyer.csv
Saved ../Data/process/Tube Mill.csv
